In [1]:
import pandas as pd
import spacy
from quote_extractor import QuoteExtractor
from dataset_config import BASE_FAKESPEAK_CONFIG, BASE_MISINFOTEXT_CONFIG
from helpers import get_groups, make_output_path, make_output_path_for_type

In [ ]:
fakespeak_config = BASE_FAKESPEAK_CONFIG
misinfotext_config = BASE_MISINFOTEXT_CONFIG

In [3]:
using_dataset = misinfotext_config

In [4]:
dataset_df = pd.read_excel(
    using_dataset["input_path"], 
    sheet_name=using_dataset["sheet_name"], 
    usecols=using_dataset["usecols"])
dataset_df.head()

,factcheckURL,originalURL,originalBodyText,originalHeadline,originalTextType,originalDate,originalDateYear
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016
1,http://www.politifact.com/california/statement...,https://users.focalbeam.com/fs/distribution:wl...,"Sacramento, CA - United States Senator Dianne ...",U.S. Senator Dianne Feinstein Opposes Prop. 64...,Press release,2016-07-12,2016
2,http://www.politifact.com/california/statement...,http://www.sacbee.com/opinion/op-ed/soapbox/ar...,We should anticipate black and gray markets in...,Why you should buy a locking gasoline cap,News and blog,2017-08-04,2017
3,http://www.politifact.com/california/statement...,https://nocagastax.com/california-gas-tax-hike...,As a ballot initiative calling for repeal of a...,California Gas-Tax-Hike Repeal Campaign Heats Up,News and blog,2017-06-15,2017
4,http://www.politifact.com/california/statement...,https://chu.house.gov/media-center/press-relea...,"WASHINGTON, DC The House of Representatives t...","Rep. Chu Decries ""Heartless"" ACA Repeal Vote",Press release,2017-05-04,2017


In [5]:
quote_extractor = QuoteExtractor("en_core_web_lg", "./config/quote_verb_list.txt")

c:\Users\Adam\AppData\Local\Python\pythoncore-3.11-64\Lib\site-packages\coreferee\manager.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [6]:
quote_annotations = quote_extractor.run_multiple(
    dataset_df[using_dataset["id_col"]], 
    dataset_df[using_dataset["text_col"]]
)

Preprocessing texts...
Creating spacy docs...
Extracting quotes...
Done extracting quotes


The resulting columns will contain a list of quotes (and the below stats) for each article.
- `quotes_num_tokens`: the number of tokens in each quote
- `proportions_of_total_tokens`: the number of tokens divided by total article tokens for each quote
- `proportion_quote_tokens_to_total_text`: the sum of number of tokens in quotes divided by total article tokens, i.e. "how much of this article is made up of quotes?"

In [7]:
dataset_df["quotes"] = [[doc["quote"] for doc in docs] 
                        for docs in quote_annotations]
dataset_df["quotes_num_tokens"] = [[doc["quote_token_count"] for doc in docs] 
                        for docs in quote_annotations]
dataset_df["proportions_of_total_tokens"] = [[doc["proportion_of_total_tokens"] for doc in docs] 
                        for docs in quote_annotations]
dataset_df["proportion_quote_tokens_to_total_text"] = dataset_df["proportions_of_total_tokens"].apply(sum)

dataset_df.head()

,factcheckURL,originalURL,originalBodyText,originalHeadline,originalTextType,originalDate,originalDateYear,quotes,quotes_num_tokens,proportions_of_total_tokens,proportion_quote_tokens_to_total_text
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,"[, abandoned pet rates have sky-rocketed in Te...","[24, 24, 19, 50, 11]","[0.041025641025641026, 0.041025641025641026, 0...",0.218803
1,http://www.politifact.com/california/statement...,https://users.focalbeam.com/fs/distribution:wl...,"Sacramento, CA - United States Senator Dianne ...",U.S. Senator Dianne Feinstein Opposes Prop. 64...,Press release,2016-07-12,2016,[that deaths in marijuana-related car crashes ...,"[15, 63]","[0.06696428571428571, 0.28125]",0.348214
2,http://www.politifact.com/california/statement...,http://www.sacbee.com/opinion/op-ed/soapbox/ar...,We should anticipate black and gray markets in...,Why you should buy a locking gasoline cap,News and blog,2017-08-04,2017,[Organized crime gangs are buying hundreds or ...,"[22, 4, 32, 8, 24, 18, 23]","[0.05104408352668213, 0.009280742459396751, 0....",0.303944
3,http://www.politifact.com/california/statement...,https://nocagastax.com/california-gas-tax-hike...,As a ballot initiative calling for repeal of a...,California Gas-Tax-Hike Repeal Campaign Heats Up,News and blog,2017-06-15,2017,[hes going to raise their gas taxes to the hig...,"[23, 16, 15, 16, 30, 19, 47]","[0.04572564612326044, 0.03180914512922465, 0.0...",0.330020
4,http://www.politifact.com/california/statement...,https://chu.house.gov/media-center/press-relea...,"WASHINGTON, DC The House of Representatives t...","Rep. Chu Decries ""Heartless"" ACA Repeal Vote",Press release,2017-05-04,2017,[],[],[],0.000000


Create a new DF that has one quote on each row.

In [8]:
all_quotes_df = dataset_df.explode(["quotes", "quotes_num_tokens", "proportions_of_total_tokens"])\
    .rename(columns={"quotes": "quote"})\
    .rename(columns={"quotes_num_tokens": "num_tokens"})\
    .rename(columns={"proportions_of_total_tokens": "proportion_of_total_tokens"})
all_quotes_df = all_quotes_df[all_quotes_df["quote"].notna()]

all_quotes_df.head()

,factcheckURL,originalURL,originalBodyText,originalHeadline,originalTextType,originalDate,originalDateYear,quote,num_tokens,proportion_of_total_tokens,proportion_quote_tokens_to_total_text
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,", abandoned pet rates have sky-rocketed in Tex...",24,0.041026,0.218803
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,", pet owners within Texas, Arizona, and Missou...",24,0.041026,0.218803
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,"a state funded, mandatory ‘pet registration’ p...",19,0.032479,0.218803
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,", an incentive program may be implemented to e...",50,0.08547,0.218803
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,"""domestic animal the size of a cat or larger""",11,0.018803,0.218803


In [9]:
years, years_dfs = get_groups(all_quotes_df, using_dataset["year_col"])
years_dfs[0].head()

,factcheckURL,originalURL,originalBodyText,originalHeadline,originalTextType,originalDate,originalDateYear,quote,num_tokens,proportion_of_total_tokens,proportion_quote_tokens_to_total_text
428,http://www.politifact.com/truth-o-meter/statem...,https://www.johnmccain.com/Informing/News/Pres...,"""On Monday, Senator Clinton told an audience a...",Statement By John McCain on Hillary Clinton,Press release,2007-08-23,2007,that the surge of troops in Iraq was 'working,10,0.066225,0.721854
428,http://www.politifact.com/truth-o-meter/statem...,https://www.johnmccain.com/Informing/News/Pres...,"""On Monday, Senator Clinton told an audience a...",Statement By John McCain on Hillary Clinton,Press release,2007-08-23,2007,the surge 'has failed' and that we should 'beg...,18,0.119205,0.721854
428,http://www.politifact.com/truth-o-meter/statem...,https://www.johnmccain.com/Informing/News/Pres...,"""On Monday, Senator Clinton told an audience a...",Statement By John McCain on Hillary Clinton,Press release,2007-08-23,2007,"""The fact that the New York senator can revers...",81,0.536424,0.721854


In [10]:
types, types_dfs = get_groups(all_quotes_df, using_dataset["type_col"])
types_dfs[0].head()

,factcheckURL,originalURL,originalBodyText,originalHeadline,originalTextType,originalDate,originalDateYear,quote,num_tokens,proportion_of_total_tokens,proportion_quote_tokens_to_total_text
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,", abandoned pet rates have sky-rocketed in Tex...",24,0.041026,0.218803
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,", pet owners within Texas, Arizona, and Missou...",24,0.041026,0.218803
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,"a state funded, mandatory ‘pet registration’ p...",19,0.032479,0.218803
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,", an incentive program may be implemented to e...",50,0.08547,0.218803
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016,"""domestic animal the size of a cat or larger""",11,0.018803,0.218803


## Writing dataframes to excel spreadsheet
There are three parts to the spreadsheet:
- The `year` sheets contain one quote on each line
    - Column `proportion_of_total_tokens` represents the number of tokens in the individual quote divided by the total number of quotes in the doc for its corresponding article
- The `year_proportion` sheets contain one article on each line
    - Column `proportion_quote_tokens_to_total_text` represents the sum of the quote tokens divided by the total number of quotes in the doc, i.e. "how much of this article is made up of quotes?"
- The `Proportion_Summary` sheet shows summary statistics for each year's `proportion_quote_tokens_to_total_text`

In [11]:
def get_quote_proportion_summary_df(dfs: list[pd.DataFrame]):
    return pd.DataFrame(
        [df["proportion_quote_tokens_to_total_text"].convert_dtypes().describe() for df in dfs],
        index=pd.Index(data=years, name="year")
    )

In [ ]:
def save_years(writer: pd.ExcelWriter, years: list[int], years_dfs: list[pd.DataFrame]):
    proportion_dfs: list[pd.DataFrame] = []
    
    for year, df in zip(years, years_dfs):
        # Save individual quotes
        df.to_excel(
            writer,
            sheet_name=str(year),
            index=False,
            columns=[using_dataset["id_col"], "quote", "num_tokens", "proportion_of_total_tokens"]
        )

        # Save proportion of quote tokens to total tokens in separate sheet
        proportion_df = df.drop_duplicates(subset=using_dataset["id_col"])
        proportion_df.to_excel(
            writer,
            sheet_name=f"{str(year)}_proportion",
            index=False,
            columns=[using_dataset["id_col"], "proportion_quote_tokens_to_total_text"]
        )

        proportion_dfs.append(proportion_df)

    get_quote_proportion_summary_df(proportion_dfs).to_excel(writer, sheet_name="Proportion Summary")

In [13]:
output_path = make_output_path(using_dataset, "quotes")

writer = pd.ExcelWriter(output_path, engine="xlsxwriter")
save_years(writer, years, years_dfs)
writer.close()

In [14]:
for type, df in zip(types, types_dfs):
    years, years_dfs = get_groups(df, using_dataset["year_col"])

    output_path = make_output_path_for_type(using_dataset, type, "quotes")

    writer = pd.ExcelWriter(output_path, engine="xlsxwriter")
    save_years(writer, years, years_dfs)
    writer.close()